In [ ]:


!pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d donkeys/retinopathy-train-2015 -p /content

import zipfile
with zipfile.ZipFile('/content/retinopathy-train-2015.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

print("Dataset downloaded and extracted to /content")


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/donkeys/retinopathy-train-2015
License(s): unknown
100% 33.8G/33.8G [16:59<00:00, 18.1MB/s]
100% 33.8G/33.8G [16:59<00:00, 35.6MB/s]
Dataset downloaded and extracted to /content


In [2]:
!pip install timm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.debug.metrics as met
import torch_xla.distributed.xla_multiprocessing as xmp
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset, random_split
import timm
import pandas as pd
from PIL import Image
import os

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class DiabeticRetinopathyDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx, 0] + ".png")
        image = Image.open(img_path).convert("RGB")
        label = int(self.data.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)
        return image, label

def trim_dataset(dataset, batch_size):
    trim_size = (len(dataset) // batch_size) * batch_size
    return Subset(dataset, range(trim_size))

batch_size = 64
val_split = 0.2 

full_dataset = DiabeticRetinopathyDataset("/content/trainLabels.csv", "/content/rescaled_train_896/rescaled_train_896", transform)
train_size = int((1 - val_split) * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataset = trim_dataset(train_dataset, batch_size)
val_dataset = trim_dataset(val_dataset, batch_size)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

def get_model():
    model = timm.create_model("seresnext50_32x4d", pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, 5) 
    return model

def train(rank):
    device = xm.xla_device()
    model = get_model().to(device)
    checkpoint_path = "/content/dr_seresnext50_2015_tpu (4).pth"
    if os.path.exists(checkpoint_path):
        model.load_state_dict(torch.load(checkpoint_path))
        print("Loaded checkpoint successfully!")

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 5  
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0

        loader = pl.MpDeviceLoader(train_loader, device)
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            xm.optimizer_step(optimizer)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        print(f"Epoch {epoch+1}/5, Loss: {total_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")

        xm.save(model.state_dict(), "dr_seresnext50_2015_tpu.pth")

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        loader = pl.MpDeviceLoader(val_loader, device)
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    print(f"Validation Accuracy: {100*correct/total:.2f}%")

if __name__ == "__main__":
    xmp.spawn(train, args=(), nprocs=1, start_method="fork")


/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/111M [00:00<?, ?B/s]

<ipython-input-3-49915cdd5702>:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


Loaded checkpoint successfully!
Epoch 1/5, Loss: 0.2264, Accuracy: 93.85%
Epoch 2/5, Loss: 0.0697, Accuracy: 97.90%
Epoch 3/5, Loss: 0.0475, Accuracy: 98.49%
Epoch 4/5, Loss: 0.0541, Accuracy: 98.20%
Epoch 5/5, Loss: 0.0449, Accuracy: 98.53%
Validation Accuracy: 91.51%


In [ ]:
import torch
import torch.nn as nn
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
from torch.utils.data import DataLoader
import timm
import os
from sklearn.metrics import classification_report

def get_model():
    model = timm.create_model("seresnext50_32x4d", pretrained=False)  
    model.fc = nn.Linear(model.fc.in_features, 5)  
    return model

batch_size = 64
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)  

device = xm.xla_device()
model = get_model().to(device)

checkpoint_path = "/content/dr_seresnext50_2015_tpu.pth" 
if os.path.exists(checkpoint_path):   
    state_dict = torch.load(checkpoint_path)   
    new_state_dict = {}
    for k, v in state_dict.items():
        new_k = k.replace('xla:0', 'cpu')
        new_state_dict[new_k] = v

    model.load_state_dict(new_state_dict)
    print("Model loaded successfully!")


model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    loader = pl.MpDeviceLoader(val_loader, device)
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

report = classification_report(all_labels, all_preds, target_names=["No DR", "Mild", "Moderate", "Severe", "Proliferative DR"])
print(report)


<ipython-input-8-b193363aace3>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(checkpoint_path)


Model loaded successfully!
                  precision    recall  f1-score   support

           No DR       0.95      0.97      0.96      5152
            Mild       0.81      0.63      0.71       496
        Moderate       0.81      0.83      0.82      1044
          Severe       0.82      0.65      0.72       142
Proliferative DR       0.92      0.86      0.89       142

        accuracy                           0.92      6976
       macro avg       0.86      0.79      0.82      6976
    weighted avg       0.91      0.92      0.91      6976

